<a href="https://colab.research.google.com/github/kKn00077/techit-diary-final-project/blob/main/ai_model/emotion_diary_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 파인튜닝

In [ ]:
# 파인튜닝 데이터 불러오기 : NSMC
# https://github.com/e9t/nsmc
import numpy as np
import pandas as pd

# 구글 공유링크를 통해 데이터를 제공합니다.
import gdown

!gdown --fuzzy "https://docs.google.com/spreadsheets/d/e/2PACX-1vSoMx0cHiyozQnOWeC6-a_LuvbHIpmplrUZIPfC3QOi_RSag8AwaoaL_rWWr4q8k42tGCn-PgMOSpoe/pub?gid=0&single=true&output=csv" -O my_sheet.csv

df = pd.read_csv('my_sheet.csv')
display(df.head(), df.shape)
display(df.value_counts('emotion'))

Downloading...
From: https://docs.google.com/spreadsheets/d/e/2PACX-1vSoMx0cHiyozQnOWeC6-a_LuvbHIpmplrUZIPfC3QOi_RSag8AwaoaL_rWWr4q8k42tGCn-PgMOSpoe/pub?gid=0&single=true&output=csv
To: /content/my_sheet.csv
11.5MB [00:01, 7.81MB/s]


,created_date,source,context,emotion
0,2015-12-31,nsmc_train,아 더빙.. 진짜 짜증나네요 목소리,화남(Anger)
1,2015-12-31,nsmc_train,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,까칠함(Disgust)
2,2015-12-31,nsmc_train,너무재밓었다그래서보는것을추천한다,기쁨(Joy)
3,2015-12-31,nsmc_train,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,따분함(Ennui)
4,2015-12-31,nsmc_train,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,기쁨(Joy)


(91104, 4)

,count
emotion,
기쁨(Joy),27571
화남(Anger),12119
슬픔(Sadness),10632
까칠함(Disgust),8956
따분함(Ennui),8462
추억(Nostalgia),6599
중립(Neutral),6013
불안함(Anxiety),4405
당황함(Embarrassment),3731


In [ ]:
import re

# 전처리: 숫자, 링크, 이모티콘 및 모든 특수문자 제거
def preprocess(text):
    if not isinstance(text, str):
        return ''  # 문자열이 아니면 빈 문자열로 반환

    # 이모티콘 및 유니코드 특수문자 제거하는 정규식
    emoji_pattern = re.compile(
        r"["
        r"\U0001F600-\U0001F64F"  # 이모티콘 범위 (유니코드)
        r"\U0001F300-\U0001F5FF"  # 기호 및 아이콘
        r"\U0001F680-\U0001F6FF"  # 운송 및 기계 관련 유니코드
        r"\U0001F700-\U0001F77F"  # 기타 유니코드 기호
        r"\U0001F780-\U0001F7FF"  # 추가 유니코드 기호
        r"\U0001F800-\U0001F8FF"  # 유니코드 기술 문자
        r"\U0001F900-\U0001F9FF"  # 제스처와 감정 관련 이모티콘
        r"\U0001FA00-\U0001FA6F"  # 이모티콘 확장
        r"\U0001FB00-\U0001FBFF"  # 추가 확장 이모티콘
        r"\u2600-\u26FF"          # 기호 및 도형 (예: ♠, ♣)
        r"\u2700-\u27BF"          # 기타 기호 (예: ✨, ✋)
        r"]+",
        flags=re.UNICODE
    )

    # 링크 제거 정규식
    link_pattern = re.compile(r'https?://\S+')

    # 숫자 제거 정규식
    text = re.sub(r'\d+', '', text)

    # 특수문자 제거 정규식 (모든 특수문자)
    text = re.sub(r'\W+', ' ', text)  # 특수문자를 공백으로 대체

    # 링크 제거
    text = link_pattern.sub('', text)

    # 이모티콘 및 유니코드 특수문자 제거
    text = emoji_pattern.sub(r'', text)

    # 양쪽 공백 제거
    return text.strip()

# 모든 텍스트에 대해 전처리 적용
df = df.copy()  # 경고 방지: 원본 데이터 복사
df['processed_context'] = df['context'].apply(preprocess)

In [ ]:
# 레이블 인코딩 (기쁨 -> 0, 슬픔-> 1 등)
label_mapping = { '기쁨(Joy)': 0, '슬픔(Sadness)' : 1, '화남(Anger)': 2, '까칠함(Disgust)': 3, '두려움(Fear)': 4, '불안함(Anxiety)': 5,
                 '부러움(Envy)': 6, '따분함(Ennui)' : 7, '당황함(Embarrassment)' : 8, '추억(Nostalgia)' : 9, '중립(Neutral)' : 10 }

# emotion 열을 숫자로 매핑
df['labels'] = df['emotion'].map(label_mapping)

display(df.head(), df.shape)

,created_date,source,context,emotion,processed_context,labels
0,2015-12-31,nsmc_train,아 더빙.. 진짜 짜증나네요 목소리,화남(Anger),아 더빙 진짜 짜증나네요 목소리,2
1,2015-12-31,nsmc_train,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,까칠함(Disgust),흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,3
2,2015-12-31,nsmc_train,너무재밓었다그래서보는것을추천한다,기쁨(Joy),너무재밓었다그래서보는것을추천한다,0
3,2015-12-31,nsmc_train,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,따분함(Ennui),교도소 이야기구먼 솔직히 재미는 없다 평점 조정,7
4,2015-12-31,nsmc_train,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,기쁨(Joy),사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,0
...,...,...,...,...,...,...
91099,2015-12-31,nsmc_train,수천년간 수없이 많은 사람들의 희생으로 구축해온 사법체계를 무너뜨릴 수는 없잖아?,슬픔(Sadness),수천년간 수없이 많은 사람들의 희생으로 구축해온 사법체계를 무너뜨릴 수는 없잖아,1
91100,2015-12-31,nsmc_train,아들 찾으러 간 다른세계(영계?)가 꿈에 나 올까 무섭다. 이승과 저승 사이 중간의...,두려움(Fear),아들 찾으러 간 다른세계 영계 가 꿈에 나 올까 무섭다 이승과 저승 사이 중간의 공...,4
91101,2015-12-31,nsmc_train,너무 재밌어요!드라마는 모두 전하고 싶은 메시지가 있죠!!!!아역들은 연기들도 잘하...,기쁨(Joy),너무 재밌어요 드라마는 모두 전하고 싶은 메시지가 있죠 아역들은 연기들도 잘하고 복...,0
91102,2015-12-31,nsmc_train,정말 당시에 최고의 액션배우라고 생각했었습니다. 정말 재밌게 봤던 영화...,추억(Nostalgia),정말 당시에 최고의 액션배우라고 생각했었습니다 정말 재밌게 봤던 영화,9


In [ ]:
# 필요한 라이브러리 설치
# !pip install torch transformers sentencepiece datasets evaluate gcsfs

In [ ]:
# KcELECTRA 모델을 NSMC(Naver Sentiment Movie Corpus)로 파인튜닝하기

# 라이브러리 임포트
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import EarlyStoppingCallback

# 모델 이름 설정 및 토크나이저 로드
model_name = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=11)

# NSMC 데이터셋 로드 (trust_remote_code 인자 추가)
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

# 데이터 전처리 함수 정의
def preprocess_function(examples):
    return tokenizer(examples['processed_context'], truncation=True, padding='max_length', max_length=512)

# 데이터셋 변환
train_dataset = Dataset.from_pandas(train_df[['processed_context', 'labels']])
test_dataset = Dataset.from_pandas(test_df[['processed_context', 'labels']])

# 토크나이즈 적용
train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

# 불필요한 컬럼 제거
train_dataset = train_dataset.remove_columns(['processed_context'])
test_dataset = test_dataset.remove_columns(['processed_context'])

# 데이터셋 형식 설정
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# 평가 지표 로드 (evaluate 라이브러리 사용)
metric = evaluate.load('accuracy')

# 평가 지표 계산 함수 정의
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return metric.compute(predictions=predictions, references=labels)

# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    run_name='my_experiment',
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=30,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    learning_rate=3e-5,
    warmup_steps=300,
    gradient_accumulation_steps=2,
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model='accuracy',
    greater_is_better=True
)

# EarlyStopping 설정
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,  # 검증 성능이 개선되지 않아도 기다릴 최대 에폭 수
    early_stopping_threshold=0.001  # 개선으로 간주할 최소 변화량
)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]  # EarlyStoppingCallback 추가
)

trainer.train()

# 학습된 모델 저장
trainer.save_model('./fine-tuned-model')
tokenizer.save_pretrained('./fine-tuned-model')

print("모델 학습이 완료되었습니다. 최적의 모델이 'fine-tuned-model' 디렉토리에 저장되었습니다.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/450k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,0.816300,0.838557,0.721572
2,0.604200,0.784165,0.741329
3,0.482500,0.844692,0.741439
4,0.261600,0.987001,0.739207
5,0.156700,1.210455,0.733975


모델 학습이 완료되었습니다. 최적의 모델이 'fine-tuned-model' 디렉토리에 저장되었습니다.


In [ ]:
# 파인튜닝한 모델 압축하여 로컬컴퓨터에 저장
!zip -r fine-tuned-model.zip ./fine-tuned-model

from google.colab import files
files.download('fine-tuned-model.zip')

  adding: fine-tuned-model/ (stored 0%)
  adding: fine-tuned-model/training_args.bin (deflated 51%)
  adding: fine-tuned-model/tokenizer_config.json (deflated 75%)
  adding: fine-tuned-model/vocab.txt (deflated 52%)
  adding: fine-tuned-model/model.safetensors (deflated 7%)
  adding: fine-tuned-model/special_tokens_map.json (deflated 42%)
  adding: fine-tuned-model/tokenizer.json (deflated 70%)
  adding: fine-tuned-model/config.json (deflated 59%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 2030여성타겟 데이터 불러오기 GO!

In [ ]:
# 자체 제작 감정 라벨 분류 데이터 불러오기
import numpy as np
import pandas as pd

# 구글 공유링크를 통해 데이터를 제공합니다.
import gdown

# 구글 드라이브 파일의 ID
file_id = '10QEZLDFyY8JwBW5PLYf44hH5gfduwtwi'
# 파일 다운로드 링크 생성
download_url = f"https://drive.google.com/uc?id={file_id}"

# 파일 다운로드
output = 'team1_emotion_data.csv'
gdown.download(download_url, output, quiet=False)

# 파일을 판다스 데이터프레임으로 로드
df = pd.read_csv(output)
display(df.head(), df.shape)
display(df.value_counts('emotion'))

Downloading...
From: https://drive.google.com/uc?id=10QEZLDFyY8JwBW5PLYf44hH5gfduwtwi
To: /content/team1_emotion_data.csv
100%|██████████| 19.8M/19.8M [00:00<00:00, 134MB/s]


,no,created_date,source,context,emotion
0,1,2024-02-03,ko-emotion-dataset,보는동안 너무 행복했고 초콜렛이 너무 먹고싶었고 티모시가 잘생겼고 울어!!하는부분이...,기쁨(Joy)
1,2,2024-02-07,ko-emotion-dataset,어릴 때 가 보고 빕스는 거의 처음인데(기억에 없음) 지금 딸기축제 기간이라 만족스...,기쁨(Joy)
2,3,2024-02-08,ko-emotion-dataset,미리 계좌로 환전해둔 돈을 해외에서 환전수수료 없이 인출 가능한 트레블로그라는 카드...,기쁨(Joy)
3,4,2024-02-09,ko-emotion-dataset,요즘 번아웃도 자꾸 올라오고 무기력해서 종강하고 교류하기도 버거운 상태가 와부렀으요ㅠㅠ,슬픔(Sadness)
4,5,2024-02-10,ko-emotion-dataset,크라임씬 장똥민이 범행 도구 찾으려고 화장실 탱크 뒤지는데 거기에 진짜 똥 넣어놓은...,기쁨(Joy)


(32934, 5)

,count
emotion,
기쁨(Joy),6336
불안함(Anxiety),5546
화남(Anger),5446
슬픔(Sadness),5007
중립(Neutral),3472
까칠함(Disgust),2008
부러움(Envy),1484
당황함(Embarrassment),1464
추억(Nostalgia),1041


In [ ]:
import re

# 전처리: 숫자, 링크, 이모티콘 및 모든 특수문자 제거
def preprocess(text):
    if not isinstance(text, str):
        return ''  # 문자열이 아니면 빈 문자열로 반환

    # 이모티콘 및 유니코드 특수문자 제거하는 정규식
    emoji_pattern = re.compile(
        r"["
        r"\U0001F600-\U0001F64F"  # 이모티콘 범위 (유니코드)
        r"\U0001F300-\U0001F5FF"  # 기호 및 아이콘
        r"\U0001F680-\U0001F6FF"  # 운송 및 기계 관련 유니코드
        r"\U0001F700-\U0001F77F"  # 기타 유니코드 기호
        r"\U0001F780-\U0001F7FF"  # 추가 유니코드 기호
        r"\U0001F800-\U0001F8FF"  # 유니코드 기술 문자
        r"\U0001F900-\U0001F9FF"  # 제스처와 감정 관련 이모티콘
        r"\U0001FA00-\U0001FA6F"  # 이모티콘 확장
        r"\U0001FB00-\U0001FBFF"  # 추가 확장 이모티콘
        r"\u2600-\u26FF"          # 기호 및 도형 (예: ♠, ♣)
        r"\u2700-\u27BF"          # 기타 기호 (예: ✨, ✋)
        r"]+",
        flags=re.UNICODE
    )

    # 링크 제거 정규식
    link_pattern = re.compile(r'https?://\S+')

    # 숫자 제거 정규식
    text = re.sub(r'\d+', '', text)

    # 특수문자 제거 정규식 (모든 특수문자)
    text = re.sub(r'\W+', ' ', text)  # 특수문자를 공백으로 대체

    # 링크 제거
    text = link_pattern.sub('', text)

    # 이모티콘 및 유니코드 특수문자 제거
    text = emoji_pattern.sub(r'', text)

    # 양쪽 공백 제거
    return text.strip()

# 모든 텍스트에 대해 전처리 적용
df['processed_context'] = df['context'].apply(preprocess)

In [ ]:
# 지나치게 긴 문장 제거
# 토큰 길이를 기반으로 필터링
max_len = 500  # 최대 토큰 길이 설정

# 각 문장에 대해 토큰화 후 길이 계산
df['context_length'] = df['context'].apply(lambda x: len(tokenizer.tokenize(str(x))))

# 길이가 max_len을 초과하는 행 제거
df = df[df['context_length'] <= max_len].drop(columns=['context_length'])

print("필터링 후 데이터 크기:", len(df))


Token indices sequence length is longer than the specified maximum sequence length for this model (564 > 512). Running this sequence through the model will result in indexing errors


필터링 후 데이터 크기: 31917


In [ ]:
# 레이블 인코딩 (기쁨 -> 0, 슬픔-> 1 등)
label_mapping = { '기쁨(Joy)': 0, '슬픔(Sadness)' : 1, '화남(Anger)': 2, '까칠함(Disgust)': 3, '두려움(Fear)': 4, '불안함(Anxiety)': 5,
                 '부러움(Envy)': 6, '따분함(Ennui)' : 7, '당황함(Embarrassment)' : 8, '추억(Nostalgia)' : 9, '중립(Neutral)' : 10 }

# emotion 열을 숫자로 매핑
df['labels'] = df['emotion'].map(label_mapping)

display(df.head(), df.shape)

,no,created_date,source,context,emotion,processed_context,labels
0,1,2024-02-03,ko-emotion-dataset,보는동안 너무 행복했고 초콜렛이 너무 먹고싶었고 티모시가 잘생겼고 울어!!하는부분이...,기쁨(Joy),보는동안 너무 행복했고 초콜렛이 너무 먹고싶었고 티모시가 잘생겼고 울어 하는부분이 ...,0
1,2,2024-02-07,ko-emotion-dataset,어릴 때 가 보고 빕스는 거의 처음인데(기억에 없음) 지금 딸기축제 기간이라 만족스...,기쁨(Joy),어릴 때 가 보고 빕스는 거의 처음인데 기억에 없음 지금 딸기축제 기간이라 만족스러...,0
2,3,2024-02-08,ko-emotion-dataset,미리 계좌로 환전해둔 돈을 해외에서 환전수수료 없이 인출 가능한 트레블로그라는 카드...,기쁨(Joy),미리 계좌로 환전해둔 돈을 해외에서 환전수수료 없이 인출 가능한 트레블로그라는 카드...,0
3,4,2024-02-09,ko-emotion-dataset,요즘 번아웃도 자꾸 올라오고 무기력해서 종강하고 교류하기도 버거운 상태가 와부렀으요ㅠㅠ,슬픔(Sadness),요즘 번아웃도 자꾸 올라오고 무기력해서 종강하고 교류하기도 버거운 상태가 와부렀으요ㅠㅠ,1
4,5,2024-02-10,ko-emotion-dataset,크라임씬 장똥민이 범행 도구 찾으려고 화장실 탱크 뒤지는데 거기에 진짜 똥 넣어놓은...,기쁨(Joy),크라임씬 장똥민이 범행 도구 찾으려고 화장실 탱크 뒤지는데 거기에 진짜 똥 넣어놓은...,0


(31917, 7)

# distilkobert로 데이터 증강

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import pandas as pd
from tqdm import tqdm
import torch

# 1. 모델 및 토크나이저 로드
model_name = "monologg/distilkobert"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForMaskedLM.from_pretrained(model_name, trust_remote_code=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# 2. 텍스트 증강 함수 정의
def augment_text_distilkobert(text, num_augmentations=5):
    augmented_texts = []
    if not isinstance(text, str):
        # 입력이 문자열이 아니면 무시
        return []

    for _ in range(num_augmentations):  # num_augmentations 횟수만큼 증강
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
        input_ids = inputs.input_ids.clone()

        # 랜덤 단어 마스킹
        mask_idx = torch.randint(1, input_ids.size(1) - 1, (1,)).item()
        input_ids[0, mask_idx] = tokenizer.mask_token_id

        # 모델 추론
        with torch.no_grad():
            outputs = model(input_ids)
        logits = outputs.logits

        # 마스크된 위치의 단어 예측
        predicted_id = torch.argmax(logits[0, mask_idx]).item()
        input_ids[0, mask_idx] = predicted_id

        # 증강된 텍스트 생성
        augmented_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
        augmented_texts.append(augmented_text)
    return augmented_texts

# 3. 데이터프레임 증강 함수
def augment_emotion_distilkobert(df, target_label, num_samples, start_no=100001):
    subset = df[df['emotion'] == target_label]
    augmented_texts = []
    current_no = start_no

    for _, row in tqdm(subset.iterrows(), total=len(subset), desc=f"증강 중: {target_label}"):
        try:
            # `processed_context` 사용
            augmented_text_list = augment_text_distilkobert(row['processed_context'], num_augmentations=num_samples // len(subset) + 1)
            for augmented_text in augmented_text_list[:num_samples]:
                augmented_texts.append({
                    "no": current_no,  # 증강된 데이터의 번호
                    "created_date": row['created_date'],
                    "source": row['source'],
                    "processed_context": augmented_text,
                    "emotion": row['emotion'],
                    "labels": int(row['labels'])  # 정수로 변환된 라벨
                })
                current_no += 1
        except Exception as e:
            print(f"증강 중 오류 발생: {e}")

    return pd.DataFrame(augmented_texts)

# 4. 라벨별 데이터 증강
emotion_counts = df['emotion'].value_counts()
max_count = emotion_counts.max()
augmented_data = []
start_no = 100001  # 증강 데이터의 번호 시작값

for emotion, count in emotion_counts.items():
    if count < max_count:
        num_to_generate = max_count - count
        print(f"\n라벨 {emotion}의 부족 샘플: {num_to_generate}")
        augmented_subset = augment_emotion_distilkobert(df, emotion, num_to_generate, start_no=start_no)
        augmented_data.append(augmented_subset)
        start_no += len(augmented_subset)  # 증강된 데이터 개수만큼 번호 증가

# 5. 증강 데이터 병합
if augmented_data:
    augmented_df = pd.concat(augmented_data)
    balanced_df = pd.concat([df, augmented_df]).reset_index(drop=True)
else:
    balanced_df = df

# 6. 결과 확인
print("\n원본 데이터 크기:", len(df))
print("증강 후 데이터 크기:", len(balanced_df))
print("\n증강 후 라벨 분포:\n", balanced_df['emotion'].value_counts())



라벨 불안함(Anxiety)의 부족 샘플: 947


증강 중: 불안함(Anxiety): 100%|██████████| 5321/5321 [00:32<00:00, 165.25it/s]



라벨 화남(Anger)의 부족 샘플: 1015


증강 중: 화남(Anger): 100%|██████████| 5253/5253 [00:26<00:00, 196.90it/s]



라벨 슬픔(Sadness)의 부족 샘플: 1643


...


라벨 두려움(Fear)의 부족 샘플: 5469


증강 중: 두려움(Fear): 100%|██████████| 799/799 [00:27<00:00, 28.58it/s]



라벨 따분함(Ennui)의 부족 샘플: 5965


증강 중: 따분함(Ennui): 100%|██████████| 303/303 [00:26<00:00, 11.33it/s]



원본 데이터 크기: 31917
증강 후 데이터 크기: 85857

증강 후 라벨 분포:
 emotion
불안함(Anxiety)          10642
화남(Anger)             10506
슬픔(Sadness)            9250
까칠함(Disgust)           7988
부러움(Envy)              7345
당황함(Embarrassment)     7235
추억(Nostalgia)          7000
중립(Neutral)            6868
두려움(Fear)              6392
따분함(Ennui)             6363
기쁨(Joy)                6268
Name: count, dtype: int64


In [ ]:
display(balanced_df.head())
print(balanced_df.isna().sum())

,no,created_date,source,context,emotion,processed_context,labels
0,1,2024-02-03,ko-emotion-dataset,보는동안 너무 행복했고 초콜렛이 너무 먹고싶었고 티모시가 잘생겼고 울어!!하는부분이...,기쁨(Joy),보는동안 너무 행복했고 초콜렛이 너무 먹고싶었고 티모시가 잘생겼고 울어 하는부분이 ...,0
1,2,2024-02-07,ko-emotion-dataset,어릴 때 가 보고 빕스는 거의 처음인데(기억에 없음) 지금 딸기축제 기간이라 만족스...,기쁨(Joy),어릴 때 가 보고 빕스는 거의 처음인데 기억에 없음 지금 딸기축제 기간이라 만족스러...,0
2,3,2024-02-08,ko-emotion-dataset,미리 계좌로 환전해둔 돈을 해외에서 환전수수료 없이 인출 가능한 트레블로그라는 카드...,기쁨(Joy),미리 계좌로 환전해둔 돈을 해외에서 환전수수료 없이 인출 가능한 트레블로그라는 카드...,0
3,4,2024-02-09,ko-emotion-dataset,요즘 번아웃도 자꾸 올라오고 무기력해서 종강하고 교류하기도 버거운 상태가 와부렀으요ㅠㅠ,슬픔(Sadness),요즘 번아웃도 자꾸 올라오고 무기력해서 종강하고 교류하기도 버거운 상태가 와부렀으요ㅠㅠ,1
4,5,2024-02-10,ko-emotion-dataset,크라임씬 장똥민이 범행 도구 찾으려고 화장실 탱크 뒤지는데 거기에 진짜 똥 넣어놓은...,기쁨(Joy),크라임씬 장똥민이 범행 도구 찾으려고 화장실 탱크 뒤지는데 거기에 진짜 똥 넣어놓은...,0


no                       0
created_date             0
source                   0
context              53941
emotion                  0
processed_context        0
labels                   0
dtype: int64


In [ ]:
balanced_df.to_csv('team1_emotion_data_balanced.csv',encoding='utf-8-sig', index=False)

# 베스트 모델 출격!

In [ ]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
from evaluate import load
from transformers import EarlyStoppingCallback

# 데이터 준비
train_df, test_df = train_test_split(balanced_df, test_size=0.3, random_state=123)
# 데이터를 증강했다면 : balanced_df
# 데이터를 증강 안했다면 : df

# 모델 이름 설정 및 토크나이저 로드
model_name = './fine-tuned-model'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=11)

# 데이터 전처리 함수 정의
def preprocess_function(examples):
    return tokenizer(examples['processed_context'], truncation=True, padding='max_length', max_length=128)

# 데이터셋 변환
train_dataset = Dataset.from_pandas(train_df[['processed_context', 'labels']])
test_dataset = Dataset.from_pandas(test_df[['processed_context', 'labels']])

# 토크나이즈 적용
train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

# 불필요한 컬럼 제거
train_dataset = train_dataset.remove_columns(['processed_context'])
test_dataset = test_dataset.remove_columns(['processed_context'])

# 데이터셋 형식 설정
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# 정확도 계산을 위한 metric 로드
metric = load('accuracy')

# compute_metrics 함수 정의
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir='./results',
    run_name='my_experiment',
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=30,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    learning_rate=3e-5,
    warmup_steps=300,
    gradient_accumulation_steps=2,
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model='accuracy',
    greater_is_better=True
)

# EarlyStopping 설정
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,  # 검증 성능이 개선되지 않아도 기다릴 최대 에폭 수
    early_stopping_threshold=0.001  # 개선으로 간주할 최소 변화량
)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]  # EarlyStoppingCallback 추가
)

# 모델 학습
trainer.train()

# 베스트 모델 저장
trainer.save_model('best_model')
tokenizer.save_pretrained('./best_model')

print("모델 학습이 완료되었습니다. 최적의 모델이 'best_model' 디렉토리에 저장되었습니다.")

Map:   0%|          | 0/60099 [00:00<?, ? examples/s]

Map:   0%|          | 0/25758 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
0,0.571600,0.530113,0.827588
2,0.151800,0.376497,0.914007
4,0.049500,0.479761,0.928178
6,0.019300,0.567742,0.931283
8,0.001800,0.591556,0.934156
9,0.000100,0.600784,0.934583


모델 학습이 완료되었습니다. 최적의 모델이 'best_model' 디렉토리에 저장되었습니다.


# 테스트 및 저장

In [3]:
# 테스트 데이터 불러오기
import numpy as np
import pandas as pd

# 구글 공유링크를 통해 데이터를 제공합니다.
import gdown

# 구글 드라이브 파일의 ID
file_id = '1ouAieJMe5EJ1wkPT48A93umwErFKVwaf'
# 파일 다운로드 링크 생성
download_url = f"https://drive.google.com/uc?id={file_id}"

# 파일 다운로드
output = 'test_sentences.csv'
gdown.download(download_url, output, quiet=False)

# 파일을 판다스 데이터프레임으로 로드
test = pd.read_csv(output)
display(test.head(), test.shape)
display(test.value_counts('emotion'))

Downloading...
From: https://drive.google.com/uc?id=1ouAieJMe5EJ1wkPT48A93umwErFKVwaf
To: /content/test_sentences.csv
100%|██████████| 11.9k/11.9k [00:00<00:00, 9.10MB/s]


,sentences,source,length,emotion
0,돌아올 곳이 있다는 건 우릴 얼마나 안심하게 만드는지,드라마 <웰컴투삼달리>,29,추억(Nostalgia)
1,여기서 멈추면 안 될 것 같아. 하지만 앞으로 나아가는 것도 너무 불안해. 내가 선...,드라마 <호텔 델루나>,24,불안함(Anxiety)
2,우리가 너한테 뭘 그렇게 심하게 했니?,드라마 <더 글로리>,21,화남(Anger)
3,"제발 그만해, 나 너무 무서워. 이러다가는 다 죽어!",드라마 <오징어게임>,29,두려움(Fear)
4,돈이 하나도 없는 사람과 돈이 너무 많은 사람의 공통점은 사는 게 재미가 없다는 거야.,드라마 <오징어게임>,48,따분함(Ennui)


(100, 4)

,count
emotion,
기쁨(Joy),22
중립(Neutral),22
슬픔(Sadness),10
불안함(Anxiety),7
추억(Nostalgia),7
부러움(Envy),6
화남(Anger),6
당황함(Embarrassment),5
까칠함(Disgust),5


In [4]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 모델 및 토크나이저 불러오기
model_path = "./best_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# 모델을 평가 모드로 설정
model.eval()

# 라벨 디코딩 매핑
label_decoding = {
    0: '기쁨(Joy)', 1: '슬픔(Sadness)', 2: '화남(Anger)', 3: '까칠함(Disgust)',
    4: '두려움(Fear)', 5: '불안함(Anxiety)', 6: '부러움(Envy)', 7: '따분함(Ennui)',
    8: '당황함(Embarrassment)', 9: '추억(Nostalgia)', 10: '중립(Neutral)'
}

# 테스트 문장 토크나이징
test_sentences = test['sentences'].tolist()
inputs = tokenizer(
    test_sentences,
    return_tensors="pt",
    truncation=True,
    padding="max_length",
    max_length=128
)

# 예측 수행
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=-1).tolist()

# 예측 라벨 디코딩
predicted_emotions = [label_decoding[label] for label in predicted_labels]

# test 데이터프레임에 예측값 추가
test['pred_emotion'] = predicted_emotions

# 정확도 계산
accuracy = (test['emotion'] == test['pred_emotion']).mean()

# 결과 출력
print("Accuracy:", accuracy)
for sentence, pred, true in zip(test['sentences'], test['pred_emotion'], test['emotion']):
    print(f"{sentence} ===> Pred: {pred}, True: {true}")

# emotion과 pred_emotion의 value_counts를 계산
emotion_counts = test['emotion'].value_counts()
predicted_emotion_counts = test['pred_emotion'].value_counts()

# 두 개를 합친 데이터프레임 생성
result = pd.DataFrame({
    "Emotion": emotion_counts,
    "Pred_emotion": predicted_emotion_counts
}).fillna(0)

# 예측 정확도 계산
result['Accuracy'] = (result['Pred_emotion'] / result['Emotion']).clip(upper=1).round(2)

# 'Accuracy' 열을 기준으로 내림차순 정렬
result_sorted = result.sort_values(by='Accuracy', ascending=False)

result_sorted.reset_index(inplace=True)
display(result_sorted)

Accuracy: 0.52
돌아올 곳이 있다는 건 우릴 얼마나 안심하게 만드는지 ===> Pred: 기쁨(Joy), True: 추억(Nostalgia)
여기서 멈추면 안 될 것 같아. 하지만 앞으로 나아가는 것도 너무 불안해. 내가 선택한 길이 맞는 걸까? ===> Pred: 불안함(Anxiety), True: 불안함(Anxiety)
우리가 너한테 뭘 그렇게 심하게 했니? ===> Pred: 슬픔(Sadness), True: 화남(Anger)
제발 그만해, 나 너무 무서워. 이러다가는 다 죽어! ===> Pred: 두려움(Fear), True: 두려움(Fear)
돈이 하나도 없는 사람과 돈이 너무 많은 사람의 공통점은 사는 게 재미가 없다는 거야. ===> Pred: 따분함(Ennui), True: 따분함(Ennui)
우린 깐부잖아. 깐부 사이에는 네 거 내 거가 없는 거야. ===> Pred: 기쁨(Joy), True: 기쁨(Joy)
첫사랑이란, 이루어지지 않기에 아름다운 것이다. ===> Pred: 추억(Nostalgia), True: 추억(Nostalgia)
모히또 가서 몰디브 한 잔? ===> Pred: 중립(Neutral), True: 기쁨(Joy)
동작 그만! 밑장 빼기냐? 내가 빙다리 핫바지로 보이냐? ===> Pred: 화남(Anger), True: 화남(Anger)
내가 10년 동안 울면서 후회하고 다짐했는데 꼭 그렇게 다 가져가야만 속이 후련했냐! ===> Pred: 슬픔(Sadness), True: 화남(Anger)
밥은 먹고 다니냐? ===> Pred: 중립(Neutral), True: 중립(Neutral)
아들아, 너는 계획이 다 있구나. ===> Pred: 기쁨(Joy), True: 중립(Neutral)
그대가 없는 시간은 별로 의미가 없어요. ===> Pred: 슬픔(Sadness), True: 슬픔(Sadness)
나는 너를 사랑해. 그 어떤 것도 변하지 않을 거야. ===> Pred: 기쁨(Joy), True: 기쁨(Joy)
너 

,index,Emotion,Pred_emotion,Accuracy
0,당황함(Embarrassment),5,5,1.00
1,따분함(Ennui),5,5,1.00
2,두려움(Fear),5,7,1.00
3,추억(Nostalgia),7,10,1.00
4,화남(Anger),6,8,1.00
5,중립(Neutral),22,25,1.00
6,슬픔(Sadness),10,14,1.00
7,부러움(Envy),6,5,0.83
8,기쁨(Joy),22,18,0.82
9,불안함(Anxiety),7,2,0.29


In [ ]:
# 파이널튜닝한 모델 압축하여 구글 드라이브에 저장
from google.colab import drive
drive.mount('/content/drive')

!zip -r /content/drive/MyDrive/best_model_093.zip ./best_model

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  adding: best_model/ (stored 0%)
  adding: best_model/training_args.bin (deflated 51%)
  adding: best_model/tokenizer_config.json (deflated 74%)
  adding: best_model/vocab.txt (deflated 52%)
  adding: best_model/model.safetensors (deflated 7%)
  adding: best_model/special_tokens_map.json (deflated 80%)
  adding: best_model/tokenizer.json (deflated 70%)
  adding: best_model/config.json (deflated 59%)
